Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1.

PART 1: PHPMyAdmin

You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


 

In [1]:
"""
QUESTIONS 

/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name
FROM Facilities
WHERE membercost > 0;

/* Q2: How many facilities do not charge a fee to members? */

SELECT count(name)
FROM Facilities
WHERE membercost = 0;

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE 
	membercost > 0
	AND membercost < (0.20 * monthlymaintenance);


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
FROM Facilities
WHERE facid IN (1,5);


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance,
CASE
    WHEN monthlymaintenance > 100 THEN 'expensive'
    ELSE 'cheap'
    END AS cost
FROM Facilities;


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM `Members`);


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT f.name as Facility, CONCAT(m.firstname, ' ', m.surname) AS Member
FROM `Members` as m 
INNER JOIN `Bookings` as b
ON m.memid = b.memid 
INNER JOIN `Facilities` as f
ON f.facid = b.facid 
WHERE b.facid IN (0, 1) AND b.memid != 0
ORDER BY Member;


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT starttime, f.name AS Facility, 
CASE WHEN b.memid != 0 THEN CONCAT(m.firstname, ' ', m.surname)
     ELSE m.firstname 
END AS name, 
CASE WHEN b.memid != 0 THEN b.slots * f.membercost
     ELSE b.slots * f.guestcost
END AS cost
FROM Bookings as b
INNER JOIN Members as m
ON b.memid = m.memid
INNER JOIN Facilities as f
ON f.facid = b.facid
WHERE CAST(starttime AS date) = CAST('2012-09-14' AS date) AND
CASE WHEN b.memid != 0 THEN b.slots * f.membercost
     ELSE b.slots * f.guestcost
END > 30.00
ORDER BY cost DESC;


/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT sub.starttime, sub.name AS Facility, 
CASE WHEN sub.memid != 0 THEN CONCAT(sub.firstname, ' ', sub.surname)
     ELSE sub.firstname 
END AS name, 
CASE WHEN sub.memid != 0 THEN sub.slots * sub.membercost
     ELSE sub.slots * sub.guestcost
END AS cost
FROM (SELECT b.starttime, b.facid, b.memid, b.slots, f.membercost, f.guestcost, f.name,
      m.firstname, m.surname
      FROM Bookings AS b
      INNER JOIN Members AS m
      ON b.memid = m.memid
      INNER JOIN Facilities AS f
      ON f.facid = b.facid) AS sub
WHERE CAST(starttime AS date) = CAST('2012-09-14' AS date) AND
CASE WHEN memid != 0 THEN slots * membercost
     ELSE slots * guestcost
END > 30
ORDER BY cost DESC;
"""

"\nQUESTIONS \n\n/* Q1: Some of the facilities charge a fee to members, but some do not.\nWrite a SQL query to produce a list of the names of the facilities that do. */\n\nSELECT name\nFROM Facilities\nWHERE membercost > 0;\n\n/* Q2: How many facilities do not charge a fee to members? */\n\nSELECT count(name)\nFROM Facilities\nWHERE membercost = 0;\n\n/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,\nwhere the fee is less than 20% of the facility's monthly maintenance cost.\nReturn the facid, facility name, member cost, and monthly maintenance of the\nfacilities in question. */\n\nSELECT facid, name, membercost, monthlymaintenance\nFROM Facilities\nWHERE \n\tmembercost > 0\n\tAND membercost < (0.20 * monthlymaintenance);\n\n\n/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.\nTry writing the query without using the OR operator. */\n\nSELECT *\nFROM Facilities\nWHERE facid IN (1,5);\n\n\n/* Q5: Produce a list of faciliti

PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  



In [2]:
from sqlalchemy import create_engine
import pandas as pd

In [3]:
filepath = r"C:\Users\Joseph Shire\Documents\Springboard Python Data Science\Python Scripts\springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db"
e = create_engine(r"sqlite:///"+filepath)
ec = e.connect()


QUESTIONS: /* Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! */

In [4]:
a10 = '''
SELECT name AS Facility, sum(rev) AS total_rev
FROM (
    SELECT f.name,  
    CASE WHEN
        b.memid != 0 THEN f.membercost * b.slots 
        ELSE f.guestcost * b.slots END
        AS rev
    FROM Facilities AS f 
    INNER JOIN Bookings AS b
    ON b.facid = f.facid
    )
    AS sub
GROUP BY sub.name
HAVING sum(rev) < 1000
ORDER BY total_rev;
'''


a10ex = ec.execute(a10)
a10df = pd.DataFrame(a10ex.fetchall())
a10df.columns = a10ex.keys()
print(a10df)

        Facility  total_rev
0   Table Tennis        180
1  Snooker Table        240
2     Pool Table        270


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */




In [10]:
a11 = '''
SELECT (m1.surname|| ', ' ||m1.firstname) AS Recommender, (m2.surname||', '||m2.firstname) AS Recommendee
FROM Members AS m1
INNER JOIN Members AS m2
ON m1.memid = m2.recommendedby
ORDER BY m1.surname, m1.firstname;
'''


a11ex = ec.execute(a11)
a11df = pd.DataFrame(a11ex.fetchall())
a11df.columns = a11ex.keys()
print(a11df)

           Recommender               Recommendee
0      Bader, Florence         Sarwin, Ramnaresh
1       Baker, Timothy              Coplin, Joan
2      Butters, Gerald          Genting, Matthew
3      Farrell, Jemima            Baker, Timothy
4      Farrell, Jemima             Pinker, David
5     Genting, Matthew         Rumney, Henrietta
6         Jones, David            Jones, Douglas
7     Joplette, Janice               Dare, Nancy
8     Joplette, Janice              Jones, David
9   Purview, Millicent                Hunt, John
10         Rownam, Tim               Boothe, Tim
11       Smith, Darren          Joplette, Janice
12       Smith, Darren           Butters, Gerald
13       Smith, Darren             Owen, Charles
14       Smith, Darren               Smith, Jack
15       Smith, Darren           Mackenzie, Anna
16        Smith, Tracy  Worthington-Smyth, Henry
17        Smith, Tracy        Purview, Millicent
18        Smith, Tracy            Crumpet, Erica
19    Stibbons, Pond

/* Q12: Find the facilities with their usage by member, but not guests */




In [20]:
a12 = '''
SELECT f.name, round(count(slots)/2.0,2) AS 'Hours of Usage'
FROM Facilities AS f
INNER JOIN Bookings AS b
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY f.name;
'''


a12ex = ec.execute(a12)
a12df = pd.DataFrame(a12ex.fetchall())
a12df.columns = a12ex.keys()
print(a12df)

              name  Hours of Usage
0  Badminton Court           172.0
1   Massage Room 1           210.5
2   Massage Room 2            13.5
3       Pool Table           391.5
4    Snooker Table           210.5
5     Squash Court            97.5
6     Table Tennis           192.5
7   Tennis Court 1           154.0
8   Tennis Court 2           138.0


/* Q13: Find the facilities usage by month, but not guests */

In [24]:
a13 = '''
SELECT f.name, strftime('%m', starttime) AS Month, round(count(slots)/2.0,2) AS 'Hours of Usage'
FROM Facilities AS f
INNER JOIN Bookings AS b
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY f.name, Month;
'''


a13ex = ec.execute(a13)
a13df = pd.DataFrame(a13ex.fetchall())
a13df.columns = a13ex.keys()
print(a13df)

               name Month  Hours of Usage
0   Badminton Court    07            25.5
1   Badminton Court    08            66.0
2   Badminton Court    09            80.5
3    Massage Room 1    07            38.5
4    Massage Room 1    08            76.5
5    Massage Room 1    09            95.5
6    Massage Room 2    07             2.0
7    Massage Room 2    08             4.5
8    Massage Room 2    09             7.0
9        Pool Table    07            51.5
10       Pool Table    08           136.0
11       Pool Table    09           204.0
12    Snooker Table    07            34.0
13    Snooker Table    08            77.0
14    Snooker Table    09            99.5
15     Squash Court    07            11.5
16     Squash Court    08            42.5
17     Squash Court    09            43.5
18     Table Tennis    07            24.0
19     Table Tennis    08            71.5
20     Table Tennis    09            97.0
21   Tennis Court 1    07            32.5
22   Tennis Court 1    08         